# Exploratory Data Analysis

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

## Loading data

In [ ]:
df = pd.read_csv("../data/transactions_dataset.csv", sep=";")

## Exploring data

In [ ]:
df.sort_values("date_order").head(10)

In [ ]:
df.info(verbose=True)

In [ ]:
df.describe()

Let's check if there is missing data...

In [ ]:
df.isna().mean()

In [ ]:
len(df[df.sales_net < 0]) / len(df)

Let's have a look at the different columns individually...

In [ ]:
df.order_channel.value_counts().plot(kind="bar")
plt.title("Distribution of order channels");

In [ ]:
df.branch_id.nunique()

In [ ]:
df.branch_id.value_counts().head(30).plot(kind="bar")
plt.title("Distribution of top 30 branches");

In [ ]:
df.client_id.nunique()

In [ ]:
df.client_id.value_counts().head(30).plot(kind="bar")
plt.title("Distribution of top 30 clients");

In [ ]:
df.product_id.nunique()

In [ ]:
df.product_id.value_counts().head(30).plot(kind="bar")
plt.title("Distribution of top 30 products");

In [ ]:
df["date_order"] = pd.to_datetime(df["date_order"])

In [ ]:
df.client_id.value_counts()

In [ ]:
df["price"] = df["sales_net"] / df["quantity"]

In [ ]:
import numpy as np

df["qauntity"] = np.where(
    df["quantity"] > 0, df["quantity"], -1 * df["quantity"]
)

In [ ]:
df_reduced = df.groupby(
    ["date_order", "client_id", "product_id", "order_channel"]
).agg({"sales_net": "sum", "quantity": "sum"})

In [ ]:
df_reduced[df_reduced["sales_net"] < 0]

In [ ]:
df["date_order"].max()

In [ ]:
df_pos = df[df.sales_net > 0]

In [ ]:
from datetime import timedelta

test_stamp = df_pos["date_order"].max() - timedelta(days=90)

In [ ]:
test_data = df_pos[df_pos["date_order"] >= test_stamp]

In [ ]:
df_pos

In [ ]:
# Sort data by 'client_id' and 'date_order'
df_pos.sort_values(["client_id", "date_order"], inplace=True)

In [ ]:
df_day = df_pos.drop_duplicates(["date_order", "client_id"])

# Calculate the time difference between consecutive purchases for each customer
df_day["time_since_previous_purchase"] = df_day.groupby("client_id")[
    "date_order"
].diff()

In [ ]:
df_day.head()

In [ ]:
# Calculate the average time between purchases for all customers
average_time_between_purchases = df_day["time_since_previous_purchase"].mean()
print(
    f"Average time between a customer's purchases: {average_time_between_purchases}"
)

In [ ]:
time_to_buy = (
    df_day.groupby("client_id")["time_since_previous_purchase"].max().dt.days
)
# time_to_buy = time_to_buy[(time_to_buy > 0) & (time_to_buy < 150)]

In [ ]:
churn_customers = time_to_buy[time_to_buy > 40].index

In [ ]:
time_to_buy.isna().mean()

In [ ]:
time_to_buy.hist(bins=100)

In [ ]:
client_percent = (
    time_to_buy.reset_index()
    .groupby("time_since_previous_purchase")
    .client_id.count()
)
client_percent = client_percent.reset_index().sort_values(
    "time_since_previous_purchase"
)
client_percent["client_id"] = (
    client_percent.client_id.cumsum(axis=0) / client_percent.client_id.sum()
)

In [ ]:
plt.plot("time_since_previous_purchase", "client_id", data=client_percent)
plt.xlabel("Avg. number of days since last purchase")
plt.ylabel("Percentage of clients")

In [ ]:
revenue_sum = df_pos.groupby("client_id").sales_net.sum()
revenue_sum = time_to_buy.reset_index().merge(
    revenue_sum, left_on="client_id", right_index=True, how="outer"
)
revenue_sum = revenue_sum.fillna(max(revenue_sum.time_since_previous_purchase))
revenue_sum = revenue_sum.sort_values("time_since_previous_purchase")
revenue_sum = (
    revenue_sum.groupby("time_since_previous_purchase")
    .sales_net.sum()
    .reset_index()
)
revenue_precent = revenue_sum.copy()
revenue_precent["sales_net"] = (
    revenue_precent.sales_net.cumsum(axis=0) / revenue_precent.sales_net.sum()
)
revenue_sum["sales_net"] = revenue_sum.sales_net.cumsum(axis=0)

In [ ]:
plt.plot(
    "time_since_previous_purchase",
    "sales_net",
    data=revenue_precent,
    label="Revenue",
)
plt.xlabel("Avg. number of days since last purchase")
plt.ylabel("Percentage of revenue")

In [ ]:
plt.rcParams.update({"font.size": 18})
plt.figure(figsize=(15, 6))
plt.plot(
    "time_since_previous_purchase",
    "sales_net",
    data=revenue_precent,
    label="Revenue",
    color="#03522D",
)
plt.plot(
    "time_since_previous_purchase",
    "client_id",
    data=client_percent,
    label="Client",
    color="#29BA74",
)
plt.vlines(x=60, ymin=0, ymax=0.983666, color="grey")
# plt.hlines(y=0.983666, xmin=-10, xmax=60, color="lightgrey")
# plt.hlines(y=0.757764, xmin=-10, xmax=60, color="lightgrey")
plt.xlabel("Avg. number of days since last purchase")
plt.ylabel("Percentage of revenue or client")
plt.legend()
plt.savefig("demo.png", transparent=True)

In [ ]:
(1 - 0.757764) * df_pos.client_id.nunique()

In [ ]:
client_percent[client_percent.time_since_previous_purchase == 60]

In [ ]:
1 - (
    revenue_sum[revenue_sum.time_since_previous_purchase == 60].sales_net
    / revenue_sum.sales_net.max()
)

In [ ]:
(
    revenue_sum.sales_net.max()
    - revenue_sum[revenue_sum.time_since_previous_purchase == 60].sales_net
)

In [ ]:
sales = df.groupby("client_id").sales_net.mean()
sales[sales < 10000].hist()

In [ ]:
sales[sales.index.isin(churn_customers)].sum() / sales.sum()

In [ ]:
(time_to_buy > 180).sum() / len(time_to_buy)

In [ ]:
(df.groupby("client_id").sales_net.mean() > 1000).mean()

In [ ]:
df[df.client_id == 1090388]

In [ ]:
df[
    (df.date_order == "2017-09-26")
    & (df.product_id == 121848)
    & (df.client_id == 1789606)
]

In [ ]:
temp.head()

In [ ]:
temp = df[df["sales_net"] < 0]
# ((pd.to_datetime(temp["date_invoice"]) - temp["date_order"]).value_counts() / len(temp)).head(50)

In [ ]:
import pandas as pd
from datetime import datetime, timedelta

# Filter data for the last 3 months
three_months_ago = datetime.now() - timedelta(
    days=90
)  # Assuming 30 days per month
recent_data = df[df["date_order"] >= three_months_ago]

# Identify unique customers who made purchases in the last 3 months
unique_customers = recent_data["client_id"].unique()

# List of customers who made purchases in the last 3 months
print("Customers who made purchases in the last 3 months:")
print(unique_customers)

In [ ]:
df.groupby("branch_id").sales_net.sum().reset_index().sort_values("sales_net")

In [ ]:
df[df.sales_net > 0].groupby(
    "branch_id"
).sales_net.sum().reset_index().sort_values("sales_net")

In [ ]:
df[df.sales_net > 0].groupby(
    "branch_id"
).quantity.sum().reset_index().sort_values("quantity")

In [ ]:
df.groupby("branch_id").quantity.sum().reset_index().sort_values("quantity")

In [ ]:
df[df.branch_id == 3318].product_id.value_counts().head(20)

In [ ]:
df[df.branch_id == 3318].product_id.nunique()

In [ ]:
df.date_order.nunique()

In [ ]:
df_3318 = df.loc[df.branch_id == 3318].copy()

In [ ]:
df_3318["date_order"] = pd.to_datetime(df_3318["date_order"])
df_3318["week_of_year"] = df_3318["date_order"].dt.isocalendar().week

df_3318["year"] = df_3318["date_order"].dt.year
df_3318["month"] = df_3318["date_order"].dt.month

In [ ]:
df_3318["price"] = df_3318["sales_net"] / df_3318["quantity"]
df_3318 = df_3318.drop(
    columns=["date_invoice", "client_id", "branch_id", "order_channel"]
)

In [ ]:
df_3318.head()

In [ ]:
temp = (
    df_3318.groupby(["year", "week_of_year", "product_id"])
    .agg(
        {
            "quantity": "sum",
            "price": "mean",
            "month": "first",
            "sales_net": "sum",
        }
    )
    .reset_index()
)

In [ ]:
prods = (
    temp.product_id.value_counts()[temp.product_id.value_counts() > 20]
    .reset_index()
    .product_id.unique()
)

In [ ]:
temp[temp.product_id.isin(prods)].sales_net.sum() / temp.sales_net.sum()

In [ ]:
len(df_3318[df_3318.sales_net < 0]) / len(df_3318)

In [ ]:
df_day = (
    df_3318.groupby(["date_order", "product_id"])[["sales_net", "quantity"]]
    .sum()
    .reset_index()
)

In [ ]:
df_day["year"] = df_day["date_order"].dt.year
df_day["week_of_year"] = df_day["date_order"].dt.isocalendar().week

In [ ]:
df_week = (
    df_day.groupby(["year", "week_of_year", "product_id"])
    .quantity.sum()
    .reset_index()
)

In [ ]:
df_week

In [ ]:
sum(df_week.product_id.value_counts() > 20)

In [ ]:
sum(df_day.product_id.value_counts() > 50)

In [ ]:
df_day.product_id.nunique()

In [ ]:
df_day.date_order.nunique()

In [ ]:
df_day["date_order"] = pd.to_datetime(df_day["date_order"])

In [ ]:
df_day[(df_day.product_id == 114938) & (df_day.date_order > "2019-07")].plot(
    x="date_order", y="quantity"
)

In [ ]:
df_day[(df_day.product_id == 114938) & (df_day.date_order > "2019-07")]